In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data

import random, os, pathlib, time
from tqdm import tqdm
from sklearn import datasets

In [2]:
import nflib
from nflib.flows import SequentialFlow, NormalizingFlow, ActNorm, AffineConstantFlow
import nflib.coupling_flows as icf
import nflib.inn_flow as inn
import nflib.res_flow as irf

In [3]:
import dtnnlib as dtnn

In [4]:
# device = torch.device("cuda:0")
device = torch.device("cpu")

## MNIST dataset

In [5]:
import mylibrary.datasets as datasets

In [6]:
mnist = datasets.FashionMNIST()
# mnist.download_mnist()
# mnist.save_mnist()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

# train_label = tnn.Logits.index_to_logit(train_label_)
train_size = len(train_label_)

In [7]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)
test_label = torch.LongTensor(test_label_)

In [8]:
input_size = 784
output_size = 10

In [9]:
class MNIST_Dataset(data.Dataset):
    
    def __init__(self, data, label):
        self.data = data
        self.label = label
        
#         self.label = mask.type(torch.float32).reshape(-1,1)
        self._shuffle_data_()
        
    def __len__(self):
        return len(self.data)
    
    def _shuffle_data_(self):
        randidx = random.sample(range(len(self.data)), k=len(self.data))
        self.data = self.data[randidx]
        self.label = self.label[randidx]
    
    def __getitem__(self, idx):
        img, lbl = self.data[idx], self.label[idx]
        return img, lbl

In [10]:
train_dataset = MNIST_Dataset(train_data, train_label)
test_dataset = MNIST_Dataset(test_data, test_label)

In [11]:
learning_rate = 0.0003
batch_size = 50

In [12]:
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

In [13]:
class ConnectedClassifier_Softmax(nn.Module):
    
    def __init__(self,input_dim, num_sets, output_dim, inv_temp=1):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_sets = num_sets
        self.inv_temp = nn.Parameter(torch.ones(1)*inv_temp)
        
        self.linear = nn.Linear(input_dim, num_sets)
        self.linear.bias.data *= 0
        self.linear.weight.data *= 0.1
        self.cls_weight = nn.Parameter(torch.ones(num_sets, output_dim)/output_dim)
        self.cls_confidence = None
        
        
    def forward(self, x, hard=True):
        x = self.linear(x)
        if hard:
            x = torch.softmax(-x*1e5, dim=1)
        else:
            x = torch.softmax(-x*self.inv_temp, dim=1)
        self.cls_confidence = x
        c = torch.softmax(self.cls_weight, dim=1)
#         c = self.cls_weight
        return x@c ## since both are normalized, it is also normalized

In [83]:
class ConnectedClassifier_SoftKMeans(nn.Module):
    
    def __init__(self,input_dim, num_sets, output_dim, inv_temp=1):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_sets = num_sets
        self.inv_temp = nn.Parameter(torch.ones(1)*inv_temp)
        
        self.dt = dtnn.DistanceTransformBase(784, num_sets)
        
        ## uniform values for all class
        init_val = torch.ones(num_sets, output_dim)/output_dim

        ## class repeat sequentially
#         init_val = torch.randn(num_sets, output_dim)*0.01
#         for ns in range(num_sets):
#             init_val[ns, ns%output_dim] = 10.
        
        self.cls_weight = nn.Parameter(init_val)

        self.cls_confidence = None
        
        self.move_loss = None
        
        
    def forward(self, x, hard=False):
#         self.cls_weight.data = torch.abs(self.cls_weight.data/self.cls_weight.data.sum(dim=1, keepdim=True))
        
        x_ = x[:, :self.input_dim]
        dists = self.dt(x_)
        dists = dists/np.sqrt(self.input_dim) ### correction to make diagonal of unit square 1 in nD space
        
        if hard:
            x = torch.softmax(-dists*1e5, dim=1)
        else:
            x = torch.softmax(-dists*self.inv_temp, dim=1)
        
        self.cls_confidence = x
        c = torch.softmax(self.cls_weight, dim=1)
#         c = self.cls_weight
        return x@c ## since both are normalized, it is also normalized
    
    def set_centroid_to_data_randomly(self, data_loader, model):
        indices = np.random.permutation(len(data_loader.dataset.data))[:self.dt.centers.shape[0]]
        xx = data_loader.dataset.data[indices].to(self.dt.centers.device)
        yy = data_loader.dataset.label[indices].to(self.dt.centers.device)
        yout = model(xx)
        self.dt.centers.data = yout
        
        init_val = torch.ones(self.num_sets, self.output_dim)/self.output_dim
        for ns in range(len(indices)):
            init_val[ns, yy[ns]] = 10.
        self.cls_weight.data = init_val.to(self.cls_weight.device)
        pass

In [14]:
class ST_Sigmoid(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, x):
        zz = 1/(1+torch.exp(-x))
        ctx.save_for_backward(zz)
        
        output = (x>0).type(x.dtype)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        zz, = ctx.saved_tensors
        grad_x = None
        
        if ctx.needs_input_grad[0]:
            grad_x = zz*(1-zz)*grad_output

        return grad_x

In [15]:
class DistanceBinaryClassifier(nn.Module):
    def __init__(self, input_dim, output_dim, inv_temp=1):
        super().__init__()
        self.centers = nn.Parameter(torch.rand(1, input_dim)*2-1)
        self.bias = nn.Parameter(torch.ones(1)*-0.5)
        self.inv_temp = nn.Parameter(torch.ones(1)*inv_temp)

        self.actf = nn.Sigmoid()
#         self.actf = ST_Sigmoid.apply
        self.pos_val = nn.Parameter(torch.ones(1, output_dim)*0.5)
        self.neg_val = nn.Parameter(torch.ones(1, output_dim)*0.5)
        
    def forward(self, x, hard=False):
        self.pos_val.data.abs_()
        self.pos_val.data = self.pos_val.data/self.pos_val.data.sum()
        self.neg_val.data.abs_()
        self.neg_val.data = self.neg_val.data/self.neg_val.data.sum()
        
        x = torch.norm(x-self.centers, dim=1, keepdim=True) + self.bias
        if hard:
            x = torch.sigmoid(-x*1e5)
        else:
            x = self.actf(-x*self.inv_temp)
        x = x*self.pos_val + (1-x)*self.neg_val
        return x

In [16]:
classifier = DistanceBinaryClassifier(784, 10, inv_temp=1.)

In [17]:
xx = iter(train_loader).next()[0][:5]
yout = classifier(xx)

In [18]:
yout.shape

torch.Size([5, 10])

In [19]:
actf = irf.Swish
flows = [
    ActNorm(784),
    irf.ResidualFlow(784, [784], activation=actf),
    ActNorm(784),
    irf.ResidualFlow(784, [784], activation=actf),
    ActNorm(784),
        ]

model = SequentialFlow(flows)
model = model.to(device)

In [88]:
# model = nn.Sequential(nn.Linear(784, 784, bias=False),
#                       nn.BatchNorm1d(784),
#                       nn.SELU(),
#                       nn.Linear(784, 784, bias=False),
#                       nn.BatchNorm1d(784),
#                       nn.SELU(),
#                      )

In [20]:
model.to(device)

SequentialFlow(
  (flows): ModuleList(
    (0): ActNorm()
    (1): ResidualFlow(
      (resblock): ModuleList(
        (0): Linear(in_features=784, out_features=784, bias=True)
        (1): Swish()
        (2): Linear(in_features=784, out_features=784, bias=True)
      )
    )
    (2): ActNorm()
    (3): ResidualFlow(
      (resblock): ModuleList(
        (0): Linear(in_features=784, out_features=784, bias=True)
        (1): Swish()
        (2): Linear(in_features=784, out_features=784, bias=True)
      )
    )
    (4): ActNorm()
  )
)

In [22]:
# classifier = ConnectedClassifier_SoftKMeans(784, 100, 10)
# classifier = ConnectedClassifier_Softmax(784, 10, 10)
classifier = DistanceBinaryClassifier(784, 10, inv_temp=1.)
classifier = classifier.to(device)

In [23]:
# classifier.set_centroid_to_data_randomly(train_loader, model)

## Model Train

In [31]:
# criterion = nn.NLLLoss()
# criterion = nn.CrossEntropyLoss()
def MSEClassificationLoss(output, target):
    zeros = torch.zeros_like(output)
    zeros[range(len(target)), target] = 1
    return nn.functional.mse_loss(output, zeros)
criterion = MSEClassificationLoss

optimizer = optim.Adam(list(model.parameters())+list(classifier.parameters()),
                       lr=0.0003)
# optimizer = optim.SGD(model.parameters(), lr=0.1)

print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  2466466


In [32]:
# for p in model.parameters():
#     print(torch.isnan(p).type(torch.float32).sum())

In [33]:
xx = iter(test_loader).next()[0]
xx.shape

torch.Size([50, 784])

In [34]:
model(xx.to(device))

tensor([[-8.7960e-01, -5.1781e-01,  6.8566e+00,  ...,  4.4845e-01,
          7.9428e-01,  1.8396e-01],
        [ 2.3615e+00, -1.4125e+00,  2.1103e-01,  ...,  4.1508e-02,
          4.9201e-01,  3.1429e-01],
        [-8.6264e-01,  1.3121e-01, -2.0147e-01,  ..., -3.2217e-01,
          9.3848e-01,  7.0858e-01],
        ...,
        [ 3.9195e-01,  1.0595e+00, -1.1738e-01,  ..., -2.3508e-01,
          9.1243e-01, -5.9324e-01],
        [ 1.8802e+00,  5.9760e-01,  5.0428e-02,  ..., -1.9275e-01,
          1.1123e+00, -2.3256e+00],
        [ 9.3228e-01,  8.6776e-01, -6.8902e-02,  ..., -6.6301e-03,
         -6.1297e-01,  1.5331e+00]], grad_fn=<AddBackward0>)

In [ ]:
losses = []
train_accs = []
test_accs = []
EPOCHS = 50

index = 0
for epoch in range(EPOCHS):
    train_acc = 0
    train_count = 0
    for xx, yy in tqdm(train_loader):
        xx, yy = xx.to(device), yy.to(device)
        
        yout = model(xx)
        yout = classifier(yout)    
        loss = criterion(yout, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(float(loss))

        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        train_acc += correct
        train_count += len(outputs)

    train_accs.append(float(train_acc)/train_count*100)
    train_acc = 0
    train_count = 0

    print(f'Epoch: {epoch}:{index},  Loss:{float(loss)}')
    test_count = 0
    test_acc = 0
    for xx, yy in tqdm(test_loader):
        xx, yy = xx.to(device), yy.to(device)
        with torch.no_grad():
            yout = classifier(model(xx))    
        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        test_acc += correct
        test_count += len(xx)
    test_accs.append(float(test_acc)/test_count*100)
    print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
    print()

### after each class index is finished training
print(f'\t-> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}')

  2%|▏         | 24/1200 [00:02<02:00,  9.78it/s]

In [97]:
# classifier.cls_weight

### Hard test accuracy with count per classifier

In [98]:
test_count = 0
test_acc = 0
set_count = torch.zeros(classifier.num_sets).to(device)
for xx, yy in tqdm(test_loader):
    xx, yy = xx.to(device), yy.to(device)
    with torch.no_grad():
        yout = classifier(model(xx), hard=True)
        set_indx, count = torch.unique(torch.argmax(classifier.cls_confidence, dim=1), return_counts=True) 
        set_count[set_indx] += count
    outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
    correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
    test_acc += correct
    test_count += len(xx)

print(f'Hard Test Acc:{float(test_acc)/test_count*100:.2f}%')
print(set_count.type(torch.long).tolist())

100%|██████████| 200/200 [00:00<00:00, 209.51it/s]

Hard Test Acc:79.41%
[790, 0, 0, 0, 0, 938, 594, 978, 0, 0, 0, 0, 1083, 0, 0, 0, 0, 0, 29, 0, 0, 0, 0, 11, 4, 0, 0, 1, 71, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 5, 993, 0, 27, 0, 912, 0, 0, 119, 1871, 0, 53, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 1140, 16, 0, 7, 0, 2, 0, 0, 7, 0, 0, 0, 0, 0, 20, 0, 306, 0, 0, 10, 0, 0]


### Hard train accuracy with count per classifier

In [99]:
test_count = 0
test_acc = 0
set_count = torch.zeros(classifier.num_sets).to(device)
for xx, yy in tqdm(train_loader):
    xx, yy = xx.to(device), yy.to(device)
    with torch.no_grad():
        yout = classifier(model(xx), hard=True)
        set_indx, count = torch.unique(torch.argmax(classifier.cls_confidence, dim=1), return_counts=True) 
        set_count[set_indx] += count
    outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
    correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
    test_acc += correct
    test_count += len(xx)

print(f'Hard Train Acc:{float(test_acc)/test_count*100:.2f}%')
print(set_count.type(torch.long).tolist())

100%|██████████| 1200/1200 [00:04<00:00, 240.86it/s]

Hard Train Acc:81.65%
[4783, 0, 0, 0, 0, 5765, 3534, 5887, 0, 0, 0, 0, 6331, 0, 0, 0, 3, 0, 192, 0, 0, 1, 1, 67, 18, 0, 0, 0, 431, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 2, 5939, 2, 123, 0, 5442, 0, 0, 695, 11300, 0, 383, 0, 0, 0, 11, 0, 0, 0, 0, 1, 0, 1, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 6819, 74, 0, 27, 0, 5, 0, 0, 89, 0, 0, 0, 0, 0, 124, 0, 1914, 0, 0, 20, 0, 0]


In [100]:
#### Classifiers that enclose any data
torch.count_nonzero(set_count)

tensor(35, device='cuda:0')

In [82]:
#### classifier with class representation
torch.argmax(classifier.cls_weight, dim=1)

tensor([5, 1, 4, 3, 6, 4, 4, 5, 4, 8, 7, 6, 1, 0, 8, 1, 5, 8, 4, 7, 6, 9, 5, 5,
        6, 4, 1, 3, 5, 0, 4, 3, 6, 4, 4, 0, 3, 0, 8, 6, 3, 9, 7, 1, 3, 4, 3, 5,
        4, 3, 1, 5, 5, 5, 1, 6, 8, 9, 4, 4, 9, 1, 7, 8, 4, 5, 5, 5, 9, 9, 8, 6,
        3, 3, 9, 6, 3, 7, 4, 7, 8, 9, 5, 5, 9, 0, 4, 3, 5, 8, 7, 4, 6, 4, 7, 5,
        7, 3, 0, 6], device='cuda:0')

In [154]:
# The class labels are same as that of initialized
# tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3,
#         4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7,
#         8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1,
#         2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5,
#         6, 7, 8, 9], device='cuda:0')

In [45]:
classifier.cls_weight

Parameter containing:
tensor([[0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
      

In [46]:
# torch.unique(torch.argmax(classifier.cls_confidence, dim=1), return_counts=True)

In [157]:
classifier.inv_temp

Parameter containing:
tensor([14.0282], device='cuda:0', requires_grad=True)

In [133]:
### example output per classifier
yout[5]

tensor([9.4520e-05, 1.5322e-04, 1.4976e-04, 1.6472e-04, 4.4802e-05, 1.5257e-04,
        9.9882e-01, 1.5200e-04, 1.1068e-04, 1.5288e-04], device='cuda:0')

In [134]:
asdfsdf ## to break the code

NameError: name 'asdfsdf' is not defined

### analyze per classifier accuracy

In [158]:
test_count = 0
test_acc = 0
set_count = torch.zeros(classifier.num_sets).to(device)
set_acc = torch.zeros(classifier.num_sets).to(device)
for xx, yy in tqdm(train_loader):
    xx, yy = xx.to(device), yy.to(device)
    with torch.no_grad():
        yout = classifier(model(xx), hard=True)
        
    cls_indx = torch.argmax(classifier.cls_confidence, dim=1)
    set_indx, count = torch.unique(cls_indx, return_counts=True) 
    set_count[set_indx] += count
    
    outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
    correct = (outputs == yy.data.cpu().numpy()).astype(float)
    
    ### class_index has 100 possible values
    for i, c in enumerate(correct):
        set_acc[cls_indx[i]] += c
    
#     print(set_acc.sum(), set_count.sum())
#     break
    test_acc += correct.sum()
    test_count += len(xx)

print(f'Hard Train Acc:{float(test_acc)/test_count*100:.2f}%')
print(set_count.type(torch.long).tolist())

100%|██████████| 1200/1200 [00:19<00:00, 60.60it/s]

Hard Train Acc:94.22%
[4, 70, 5920, 0, 19, 0, 12, 3, 0, 2, 5, 0, 0, 6080, 0, 0, 0, 5984, 50, 0, 1, 7, 0, 8, 1, 1, 13, 0, 0, 14, 36, 1, 0, 1, 0, 971, 1876, 5, 7, 1, 71, 0, 12, 32, 0, 0, 5819, 5, 11, 0, 162, 0, 77, 6092, 37, 6243, 5, 343, 542, 205, 7, 1, 13, 16, 3, 5572, 3846, 7, 3197, 12, 0, 1, 1, 1, 0, 0, 6, 132, 4, 0, 32, 80, 25, 11, 0, 19, 37, 1, 0, 1, 0, 188, 0, 0, 48, 0, 3, 5971, 17, 0]


In [161]:
set_acc/set_count

tensor([1.0000, 0.5714, 0.9965,    nan, 0.5789,    nan, 1.0000, 0.6667,    nan,
        0.0000, 1.0000,    nan,    nan, 0.9704,    nan,    nan,    nan, 0.9883,
        0.5200,    nan, 1.0000, 0.8571,    nan, 1.0000, 1.0000, 0.0000, 0.1538,
           nan,    nan, 0.5714, 1.0000, 1.0000,    nan, 1.0000,    nan, 1.0000,
        0.7820, 0.2000, 0.2857, 0.0000, 1.0000,    nan, 1.0000, 1.0000,    nan,
           nan, 0.9443, 0.0000, 0.5455,    nan, 0.9877,    nan, 0.9870, 0.9703,
        0.2703, 0.9031, 0.6000, 0.9883, 1.0000, 0.7171, 0.4286, 0.0000, 1.0000,
        0.5000, 1.0000, 0.9248, 0.9280, 0.7143, 0.9984, 0.3333,    nan, 1.0000,
        0.0000, 1.0000,    nan,    nan, 0.6667, 0.5076, 0.2500,    nan, 0.5625,
        0.9750, 0.2800, 0.1818,    nan, 0.8947, 1.0000, 0.0000,    nan, 0.0000,
           nan, 0.9840,    nan,    nan, 0.9167,    nan, 0.3333, 0.8970, 0.2941,
           nan], device='cuda:0')

In [162]:
print(f"Index\tNumData\tClass\tAccuracy")
for i, (cnt, acc, cls) in enumerate(zip(set_count.type(torch.long).tolist(),
                                   (set_acc/set_count).tolist(),
                                   torch.argmax(classifier.cls_weight, dim=1).tolist())):
    if cnt == 0: continue
    print(f"{i}\t {cnt}\t {cls}\t {acc*100:.2f}%")

Index	NumData	Class	Accuracy
0	 4	 5	 100.00%
1	 70	 3	 57.14%
2	 5920	 1	 99.65%
4	 19	 4	 57.89%
6	 12	 5	 100.00%
7	 3	 4	 66.67%
9	 2	 6	 0.00%
10	 5	 6	 100.00%
13	 6080	 7	 97.04%
17	 5984	 8	 98.83%
18	 50	 3	 52.00%
20	 1	 4	 100.00%
21	 7	 5	 85.71%
23	 8	 5	 100.00%
24	 1	 6	 100.00%
25	 1	 7	 0.00%
26	 13	 6	 15.38%
29	 14	 6	 57.14%
30	 36	 5	 100.00%
31	 1	 4	 100.00%
33	 1	 4	 100.00%
35	 971	 5	 100.00%
36	 1876	 6	 78.20%
37	 5	 4	 20.00%
38	 7	 6	 28.57%
39	 1	 7	 0.00%
40	 71	 5	 100.00%
42	 12	 5	 100.00%
43	 32	 5	 100.00%
46	 5819	 3	 94.43%
47	 5	 8	 0.00%
48	 11	 4	 54.55%
50	 162	 5	 98.77%
52	 77	 5	 98.70%
53	 6092	 9	 97.03%
54	 37	 6	 27.03%
55	 6243	 0	 90.31%
56	 5	 4	 60.00%
57	 343	 5	 98.83%
58	 542	 5	 100.00%
59	 205	 3	 71.71%
60	 7	 6	 42.86%
61	 1	 4	 0.00%
62	 13	 5	 100.00%
63	 16	 6	 50.00%
64	 3	 5	 100.00%
65	 5572	 2	 92.48%
66	 3846	 6	 92.80%
67	 7	 4	 71.43%
68	 3197	 5	 99.84%
69	 12	 6	 33.33%
71	 1	 6	 100.00%
72	 1	 5	 0.00%
73	 1	 6	 